In [ ]:
import csv
import json
import os
import re
from datetime import datetime

CSV_FILE = "contacts.csv"
JSON_FILE = "contacts.json"
LOG_FILE = "error_log.txt"

EMAIL_RE = re.compile(r"^[^@\s]+@[^@\s]+\.[^@\s]+$")


def log_error(message, operation):
    try:
        with open(LOG_FILE, "a", encoding="utf-8") as log:
            log.write(f"[{datetime.now()}] Operation: {operation} | Error: {message}\n")
    except Exception:
        pass


def validate_phone(phone: str) -> bool:
    s = re.sub(r"[^\d]", "", phone)
    return 7 <= len(s) <= 15 and s.isdigit()


def validate_email(email: str) -> bool:
    return bool(EMAIL_RE.match(email))


def read_contacts_from_csv() -> list:
    if not os.path.exists(CSV_FILE):
        return []
    try:
        with open(CSV_FILE, "r", encoding="utf-8", newline="") as f:
            return list(csv.DictReader(f))
    except Exception as e:
        log_error(str(e), "Read CSV")
        return []


def write_contacts_to_csv(contacts: list):
    try:
        with open(CSV_FILE, "w", encoding="utf-8", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=["name", "phone", "email"])
            writer.writeheader()
            writer.writerows(contacts)
    except Exception as e:
        log_error(str(e), "Write CSV")
        raise


def add_contact():
    try:
        name = input("Enter name: ").strip()
        if not name:
            print("Name cannot be empty.")
            return

        phone = input("Enter phone number: ").strip()
        if phone and not validate_phone(phone):
            print("Invalid phone.")
            return

        email = input("Enter email: ").strip()
        if email and not validate_email(email):
            print("Invalid email.")
            return

        contacts = read_contacts_from_csv()
        existing = next((c for c in contacts if c["name"].strip().lower() == name.lower()), None)

        if existing:
            resp = input("Contact exists. Overwrite? (y/N): ").strip().lower()
            if resp != "y":
                print("Add cancelled.")
                return
            existing["phone"] = phone
            existing["email"] = email
        else:
            contacts.append({"name": name, "phone": phone, "email": email})

        contacts = sorted(contacts, key=lambda x: x["name"].lower())
        write_contacts_to_csv(contacts)
        print("Contact saved.")

    except Exception as e:
        print("Error adding contact.")
        log_error(str(e), "Add Contact")


def display_contacts():
    try:
        contacts = read_contacts_from_csv()
        if not contacts:
            print("No contacts.")
            return

        contacts = sorted(contacts, key=lambda x: x["name"].lower())

        name_w = max(4, max((len(c["name"]) for c in contacts), default=4))
        phone_w = max(5, max((len(c["phone"]) for c in contacts), default=5))
        email_w = max(5, max((len(c["email"]) for c in contacts), default=5))

        header = f"{'Name':{name_w}}  {'Phone':{phone_w}}  {'Email':{email_w}}"
        print("\n----- CONTACT LIST -----")
        print(header)
        print("-" * len(header))

        for c in contacts:
            print(f"{c['name']:{name_w}}  {c['phone']:{phone_w}}  {c['email']:{email_w}}")
        print()

    except Exception as e:
        print("Error displaying contacts.")
        log_error(str(e), "Display Contacts")


def search_contact():
    try:
        q = input("Enter name to search: ").strip().lower()
        if not q:
            print("Empty search.")
            return

        contacts = read_contacts_from_csv()
        matches = [c for c in contacts if q in c["name"].lower()]

        if not matches:
            print("No matches.")
            return

        print(f"\nFound {len(matches)} match(es):")
        for c in matches:
            print(f"- {c['name']} | {c['phone']} | {c['email']}")
        print()

    except Exception as e:
        print("Error searching contacts.")
        log_error(str(e), "Search Contact")


def update_contact():
    try:
        q = input("Enter name to update: ").strip().lower()
        if not q:
            print("Empty name.")
            return

        contacts = read_contacts_from_csv()
        matches = [c for c in contacts if q in c["name"].lower()]

        if not matches:
            print("No contact found.")
            return

        if len(matches) > 1:
            for idx, c in enumerate(matches, 1):
                print(f"{idx}. {c['name']} | {c['phone']} | {c['email']}")
            sel = input("Choose number: ").strip()
            if not sel.isdigit() or not (1 <= int(sel) <= len(matches)):
                print("Cancelled.")
                return
            target = matches[int(sel)-1]
        else:
            target = matches[0]

        new_phone = input(f"New phone [{target['phone']}]: ").strip()
        if new_phone and not validate_phone(new_phone):
            print("Invalid phone.")
            return

        new_email = input(f"New email [{target['email']}]: ").strip()
        if new_email and not validate_email(new_email):
            print("Invalid email.")
            return

        if new_phone:
            target["phone"] = new_phone
        if new_email:
            target["email"] = new_email

        write_contacts_to_csv(contacts)
        print("Updated.")

    except Exception as e:
        print("Error updating contact.")
        log_error(str(e), "Update Contact")


def delete_contact():
    try:
        q = input("Enter name to delete: ").strip().lower()
        if not q:
            print("Empty name.")
            return

        contacts = read_contacts_from_csv()
        matches = [c for c in contacts if q in c["name"].lower()]

        if not matches:
            print("No match.")
            return

        if len(matches) > 1:
            for idx, c in enumerate(matches, 1):
                print(f"{idx}. {c['name']} | {c['phone']} | {c['email']}")
            sel = input("Choose number: ").strip()
            if not sel.isdigit() or not (1 <= int(sel) <= len(matches)):
                print("Cancelled.")
                return
            target = matches[int(sel)-1]
        else:
            target = matches[0]

        confirm = input(f"Delete {target['name']}? (y/N): ").strip().lower()
        if confirm != "y":
            print("Cancelled.")
            return

        new_contacts = [c for c in contacts if not (c["name"] == target["name"] and c["phone"] == target["phone"])]
        write_contacts_to_csv(new_contacts)
        print("Deleted.")

    except Exception as e:
        print("Error deleting contact.")
        log_error(str(e), "Delete Contact")


def export_to_json():
    try:
        contacts = read_contacts_from_csv()
        if not contacts:
            print("No contacts to export.")
            return

        with open(JSON_FILE, "w", encoding="utf-8") as jf:
            json.dump(contacts, jf, indent=4, ensure_ascii=False)

        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        backup = f"contacts_backup_{ts}.json"

        with open(backup, "w", encoding="utf-8") as bf:
            json.dump(contacts, bf, indent=4, ensure_ascii=False)

        print(f"Exported to {JSON_FILE} and {backup}.")

    except Exception as e:
        print("Error exporting JSON.")
        log_error(str(e), "Export JSON")


def load_from_json():
    try:
        if not os.path.exists(JSON_FILE):
            print("JSON file not found.")
            return

        with open(JSON_FILE, "r", encoding="utf-8") as jf:
            loaded = json.load(jf)

        print(f"Loaded {len(loaded)} contacts from JSON.")
        choice = input("Merge or overwrite CSV? (m/O): ").strip().lower()

        contacts = read_contacts_from_csv()

        if choice == "o":
            contacts = loaded
        else:
            existing_names = {c["name"].strip().lower(): c for c in contacts}
            for c in loaded:
                key = c["name"].strip().lower()
                if key in existing_names:
                    ex = existing_names[key]
                    if not ex["phone"] and c.get("phone"):
                        ex["phone"] = c["phone"]
                    if not ex["email"] and c.get("email"):
                        ex["email"] = c["email"]
                else:
                    contacts.append(c)

        contacts = sorted(contacts, key=lambda x: x["name"].lower())
        write_contacts_to_csv(contacts)
        print("Loaded into CSV.")

    except Exception as e:
        print("Error loading JSON.")
        log_error(str(e), "Load JSON")


def main():
    print("\nContact Book System\n")

    actions = {
        "1": ("Add Contact", add_contact),
        "2": ("View Contacts", display_contacts),
        "3": ("Search Contact", search_contact),
        "4": ("Update Contact", update_contact),
        "5": ("Delete Contact", delete_contact),
        "6": ("Export to JSON", export_to_json),
        "7": ("Load from JSON", load_from_json),
        "8": ("Exit", None),
    }

    while True:
        print("\nMenu:")
        for k, v in actions.items():
            print(f"{k}. {v[0]}")
        choice = input("Enter your choice: ").strip()

        if choice == "8":
            print("Goodbye!")
            break
        if choice in actions:
            if actions[choice][1]:
                actions[choice][1]()
        else:
            print("Invalid choice.")


if __name__ == "__main__":
    main()



Contact Book System


Menu:
1. Add Contact
2. View Contacts
3. Search Contact
4. Update Contact
5. Delete Contact
6. Export to JSON
7. Load from JSON
8. Exit
Enter your choice: 1
Enter name: sky
Enter phone number: 7073077606
Enter email: sk45
Invalid email.

Menu:
1. Add Contact
2. View Contacts
3. Search Contact
4. Update Contact
5. Delete Contact
6. Export to JSON
7. Load from JSON
8. Exit
Enter your choice: 2
No contacts.

Menu:
1. Add Contact
2. View Contacts
3. Search Contact
4. Update Contact
5. Delete Contact
6. Export to JSON
7. Load from JSON
8. Exit
Enter your choice: 3
Enter name to search: sky
No matches.

Menu:
1. Add Contact
2. View Contacts
3. Search Contact
4. Update Contact
5. Delete Contact
6. Export to JSON
7. Load from JSON
8. Exit
